<a href="https://colab.research.google.com/github/wishwaD7/swap-in-python/blob/master/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install "web3-ethereum-defi[data]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.6/120.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.8/99.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.9/349.9 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [25]:
# The address of a token we are going to swap out
#
# Use https://tradingstrategy.ai/search to find your token
#
# For quote terminology see https://tradingstrategy.ai/glossary/quote-token
#
import decimal
import os
import sys
from decimal import Decimal
from eth_defi.provider.multi_provider import create_multi_provider_web3
from eth_defi.uniswap_v3.constants import UNISWAP_V3_DEPLOYMENTS
from eth_defi.uniswap_v3.deployment import fetch_deployment
from eth_account import Account
from eth_account.signers.local import LocalAccount
from web3.middleware import construct_sign_and_send_raw_middleware
from eth_defi.token import fetch_erc20_details
from eth_defi.uniswap_v3.swap import swap_with_slippage_protection
from eth_defi.confirmation import wait_transactions_to_complete
import datetime
from eth_defi.revert_reason import fetch_transaction_revert_reason


QUOTE_TOKEN_ADDRESS = "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14"  # WETH (native)

# The address of a token we are going to receive
#
# Use https://tradingstrategy.ai/search to find your token
#
# For base terminology see https://tradingstrategy.ai/glossary/base-token
# BASE_TOKEN_ADDRESS = "0x2F813BDd15E9f2821545dFDfBF9BB570Fb88D041"  # USDT
BASE_TOKEN_ADDRESS = "0xf08A50178dfcDe18524640EA6618a1f965821715" #USDC

# rpc_env_var_name = "JSON_RPC_POLYGON"
# json_rpc_url = os.environ.get(rpc_env_var_name)
# assert (
#     json_rpc_url
# ), f"You need to give {rpc_env_var_name} node URL. Check ethereumnodes.com for options"

# web3 = create_multi_provider_web3("https://ethereum-sepolia-rpc.publicnode.com")
web3 = Web3(Web3.HTTPProvider('https://sepolia.infura.io/v3/00de74a63d9043f994b7cf455ee1e5af'))


print(
    f"Connected to blockchain, chain id is {web3.eth.chain_id}. the latest block is {web3.eth.block_number:,}"
)

uniswap_v3 = fetch_deployment(
    web3,
    factory_address="0x0227628f3F023bb0B980b67D528571c95c6DaC1c",
    router_address="0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E",
    position_manager_address="0x1238536071E1c677A632429e3655c799b22cDA52",
    quoter_address="0xEd1f6473345F45b75F8179591dd5bA1888cf2FB3",
)

print(f"Using Uniwap v3 compatible router at {uniswap_v3.swap_router.address}")


# Read and setup a local private key
os.environ['PRIVATE_KEY'] = '6eec9ef4c8f62e5a3432ee8dcded3da0455cdf4d2bc5b6c0d4028273b6c7d5af'

# Retrieve the private key from the environment variable
private_key = os.getenv('PRIVATE_KEY')

# Ensure the private key is in the correct format
private_key = "0x" + private_key if not private_key.startswith("0x") else private_key

# Validate the private key
assert private_key is not None, "You must set PRIVATE_KEY environment variable"
assert private_key.startswith("0x"), "Private key must start with 0x hex prefix"

# Create the account from the private key
account: LocalAccount = Account.from_key(private_key)
my_address = '0x6B0b3511756f11955ca6AA69d82790E900c58D07'

# Optionally, you can verify if the my_address matches the account address
assert account.address == my_address, "Account address does not match the provided MetaMask address"

# Now you can use `my_address` as your MetaMask wallet address
print("Your MetaMask wallet address is:", my_address)

# Enable eth_sendTransaction using this private key
web3.middleware_onion.add(construct_sign_and_send_raw_middleware(account))

# Read on-chain ERC-20 token data (name, symbol, etc.)
base = fetch_erc20_details(web3, BASE_TOKEN_ADDRESS)
quote = fetch_erc20_details(web3, QUOTE_TOKEN_ADDRESS)

# Native token balance
# See https://tradingstrategy.ai/glossary/native-token
gas_balance = web3.eth.get_balance(account.address)

print(f"Your address is {my_address}")
print(f"Your have {base.fetch_balance_of(my_address)} {base.symbol}")
print(f"Your have {quote.fetch_balance_of(my_address)} {quote.symbol}")
print(f"Your have {gas_balance / (10 ** 18)} for gas fees")

assert (
    quote.fetch_balance_of(my_address) > 0
), f"Cannot perform swap, as you have zero {quote.symbol} needed to swap"

# Ask for transfer details
decimal_amount = input(
    f"How many {quote.symbol} tokens you wish to swap to {base.symbol}? "
)

# Some input validation
try:
    decimal_amount = Decimal(decimal_amount)
except (ValueError, decimal.InvalidOperation) as e:
    raise AssertionError(f"Not a good decimal amount: {decimal_amount}") from e

# Fat-fingering check
print(f"Confirm swap amount {decimal_amount} {quote.symbol} to {base.symbol}")
confirm = input("Ok [y/n]?")
if not confirm.lower().startswith("y"):
    print("Aborted")
    sys.exit(1)

raw_amount = quote.convert_to_raw(decimal_amount)

approve = quote.contract.functions.approve(uniswap_v3.swap_router.address, raw_amount)
# print("aaaa", approve)
tx_1 = approve.build_transaction(
    {
        # approve() may take more than 500,000 gas on Arbitrum One
        "gas": 850_000,
        "from": my_address,
    }
)

print("approve tx" + str(tx_1),tx_1)

# (uniswap_v2_deployment, *, recipient_address, base_token, quote_token, intermediate_token=None, max_slippage=0.1, amount_in=None, amount_out=None, fee=30, deadline=9223372036854775808)
bound_solidity_func = swap_with_slippage_protection(
    uniswap_v3,
    recipient_address=my_address,
    base_token=base,
    quote_token=quote,
    max_slippage=20,  # Allow 40 BPS slippage before tx reverts
    amount_in=raw_amount,
    pool_fees=[500],  # 5 BPS pool WETH-USDC
    deadline=int((datetime.datetime.now() + datetime.timedelta(minutes=10)).timestamp())
)

tx_2 = bound_solidity_func.build_transaction(
    {
        # Uniswap swap should not take more than 1M gas units.
        # We do not use automatic gas estimation, as it is unreliable
        # and the number here is the maximum value only.
        # Only way to know this number is by trial and error
        # and experience.
        "gas": 1_000_000,
        "from": my_address,
    }
)

# Sign and broadcast the transaction using our private key
tx_hash_1 = web3.eth.send_transaction(tx_1)
tx_hash_2 = web3.eth.send_transaction(tx_2)

tx_wait_minutes = 2.5
print(
    f"Broadcasted transactions {tx_hash_1.hex()}, {tx_hash_2.hex()}, now waiting {tx_wait_minutes} minutes for it to be included in a new block"
)
print(
    f"View your transactions confirming at https://sepolia.etherscan.io/address/{my_address}"
)

receipts = wait_transactions_to_complete(
    web3,
    [tx_hash_1, tx_hash_2],
    max_timeout=datetime.timedelta(minutes=tx_wait_minutes),
    confirmation_block_count=1,
)

for completed_tx_hash, receipt in receipts.items():
    if receipt["status"] == 0:
        revert_reason = fetch_transaction_revert_reason(web3, completed_tx_hash)
        raise AssertionError(
            f"Our transaction {completed_tx_hash.hex()} failed because of: {revert_reason}"
        )

print("All ok!")
print(f"After swap, you have {base.fetch_balance_of(my_address)} {base.symbol}")
print(f"After swap, you have {quote.fetch_balance_of(my_address)} {quote.symbol}")
print(f"After swap, you have {gas_balance / (10 ** 18)} native token left")

Connected to blockchain, chain id is 11155111. the latest block is 6,030,600
Using Uniwap v3 compatible router at 0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E
Your MetaMask wallet address is: 0x6B0b3511756f11955ca6AA69d82790E900c58D07
Your address is 0x6B0b3511756f11955ca6AA69d82790E900c58D07
Your have 0 USDC
Your have 0.15 WETH
Your have 0.5319647180725285 for gas fees
How many WETH tokens you wish to swap to USDC? 0.01
Confirm swap amount 0.01 WETH to USDC
Ok [y/n]?y
aaaa <Function approve(address,uint256) bound to ('0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E', 10000000000000000)>
approve tx{'value': 0, 'maxFeePerGas': 16832246569, 'maxPriorityFeePerGas': 555241117, 'chainId': 11155111, 'gas': 850000, 'from': '0x6B0b3511756f11955ca6AA69d82790E900c58D07', 'to': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14', 'data': '0x095ea7b30000000000000000000000003bfa4769fb09eefc5a80d6e87c3b9c650f7ae48e000000000000000000000000000000000000000000000000002386f26fc10000'} {'value': 0, 'maxFeePerGas': 1683

ContractLogicError: execution reverted

In [14]:
import decimal
import os
import sys
from decimal import Decimal
from eth_defi.provider.multi_provider import create_multi_provider_web3
from eth_defi.uniswap_v3.constants import UNISWAP_V3_DEPLOYMENTS
from eth_defi.uniswap_v3.deployment import fetch_deployment
from eth_account import Account
from eth_account.signers.local import LocalAccount
from web3.middleware import construct_sign_and_send_raw_middleware
from eth_defi.token import fetch_erc20_details
from eth_defi.uniswap_v3.swap import swap_with_slippage_protection
from eth_defi.confirmation import wait_transactions_to_complete
import datetime
from eth_defi.revert_reason import fetch_transaction_revert_reason
from web3 import Web3

# Addresses and constants
QUOTE_TOKEN_ADDRESS = "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14"  # WETH (native)
BASE_TOKEN_ADDRESS = "0x2F813BDd15E9f2821545dFDfBF9BB570Fb88D041"  # USDT

# Connect to a different Ethereum node provider
# web3 = create_multi_provider_web3("https://mainnet.infura.io/v3/e7349ec5a1fb4cc5b196274a0416cca6")  # Replace with your Infura project ID
web3 = Web3(Web3.HTTPProvider('https://sepolia.infura.io/v3/00de74a63d9043f994b7cf455ee1e5af'))

print(f"Connected to blockchain, chain id is {web3.eth.chain_id}. the latest block is {web3.eth.block_number:,}")

# Fetch Uniswap v3 deployment details
uniswap_v3 = fetch_deployment(
    web3,
    factory_address="0x0227628f3F023bb0B980b67D528571c95c6DaC1c",
    router_address="0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E",
    position_manager_address="0x1238536071E1c677A632429e3655c799b22cDA52",
    quoter_address="0xEd1f6473345F45b75F8179591dd5bA1888cf2FB3",
)

print(f"Using Uniwap v3 compatible router at {uniswap_v3.swap_router.address}")

# Read and setup a local private key
os.environ['PRIVATE_KEY'] = '6eec9ef4c8f62e5a3432ee8dcded3da0455cdf4d2bc5b6c0d4028273b6c7d5af'
private_key = os.getenv('PRIVATE_KEY')
private_key = "0x" + private_key if not private_key.startswith("0x") else private_key

assert private_key is not None, "You must set PRIVATE_KEY environment variable"
assert private_key.startswith("0x"), "Private key must start with 0x hex prefix"

account: LocalAccount = Account.from_key(private_key)
my_address = '0x6B0b3511756f11955ca6AA69d82790E900c58D07'
assert account.address == my_address, "Account address does not match the provided MetaMask address"

web3.middleware_onion.add(construct_sign_and_send_raw_middleware(account))

base = fetch_erc20_details(web3, BASE_TOKEN_ADDRESS)
quote = fetch_erc20_details(web3, QUOTE_TOKEN_ADDRESS)
gas_balance = web3.eth.get_balance(account.address)

# print(f"Your address is {my_address}")
print("Your MetaMask wallet address is:", my_address)
print(f"Your have {base.fetch_balance_of(my_address)} {base.symbol}")
print(f"Your have {quote.fetch_balance_of(my_address)} {quote.symbol}")
print(f"Your have {gas_balance / (10 ** 18)} for gas fees")

assert quote.fetch_balance_of(my_address) > 0, f"Cannot perform swap, as you have zero {quote.symbol} needed to swap"

decimal_amount = input(f"How many {quote.symbol} tokens you wish to swap to {base.symbol}? ")

try:
    decimal_amount = Decimal(decimal_amount)
except (ValueError, decimal.InvalidOperation) as e:
    raise AssertionError(f"Not a good decimal amount: {decimal_amount}") from e

print(f"Confirm swap amount {decimal_amount} {quote.symbol} to {base.symbol}")
confirm = input("Ok [y/n]?")
if not confirm.lower().startswith("y"):
    print("Aborted")
    sys.exit(1)

raw_amount = quote.convert_to_raw(decimal_amount)
print('quote contract',quote.contract)
approve = quote.contract.functions.approve(uniswap_v3.swap_router.address, raw_amount)
tx_1 = approve.build_transaction({"gas": 850_000, "from": my_address})

print("approve tx" + str(tx_1))

bound_solidity_func = swap_with_slippage_protection(
    uniswap_v3,
    base_token=base,
    quote_token=quote,
    max_slippage=40,  # Allow 20 BPS slippage before tx reverts
    amount_in=raw_amount,
    recipient_address=my_address,
    pool_fees=[500],  # 5 BPS pool WETH-USDC
)

tx_2 = bound_solidity_func.build_transaction({"gas": 1_000_000, "from": my_address})

tx_hash_1 = web3.eth.send_transaction(tx_1)
tx_hash_2 = web3.eth.send_transaction(tx_2)

tx_wait_minutes = 2.5
print(f"Broadcasted transactions {tx_hash_1.hex()}, {tx_hash_2.hex()}, now waiting {tx_wait_minutes} minutes for it to be included in a new block")
print(f"View your transactions confirming at https://sepolia.etherscan.io/address/{my_address}")

receipts = wait_transactions_to_complete(
    web3,
    [tx_hash_1, tx_hash_2],
    max_timeout=datetime.timedelta(minutes=tx_wait_minutes),
    confirmation_block_count=1,
)

for completed_tx_hash, receipt in receipts.items():
    if receipt["status"] == 0:
        revert_reason = fetch_transaction_revert_reason(web3, completed_tx_hash)
        raise AssertionError(f"Our transaction {completed_tx_hash.hex()} failed because of: {revert_reason}")

print("All ok!")
print(f"After swap, you have {base.fetch_balance_of(my_address)} {base.symbol}")
print(f"After swap, you have {quote.fetch_balance_of(my_address)} {quote.symbol}")
print(f"After swap, you have {gas_balance / (10 ** 18)} native token left")


Connected to blockchain, chain id is 11155111. the latest block is 6,029,892
Using Uniwap v3 compatible router at 0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E
Your MetaMask wallet address is: 0x6B0b3511756f11955ca6AA69d82790E900c58D07
Your have 0 USDT
Your have 0.15 WETH
Your have 0.23208211307252855 for gas fees
How many WETH tokens you wish to swap to USDT? 0.01
Confirm swap amount 0.01 WETH to USDT
Ok [y/n]?y
quote contract <web3._utils.datatypes.Contract object at 0x786e4416fd30>
approve tx{'value': 0, 'maxFeePerGas': 12600225222116, 'maxPriorityFeePerGas': 1000000000, 'chainId': 11155111, 'gas': 850000, 'from': '0x6B0b3511756f11955ca6AA69d82790E900c58D07', 'to': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14', 'data': '0x095ea7b30000000000000000000000003bfa4769fb09eefc5a80d6e87c3b9c650f7ae48e000000000000000000000000000000000000000000000000002386f26fc10000'}


ContractLogicError: execution reverted

In [26]:
import decimal
import os
import sys
from decimal import Decimal
from eth_defi.provider.multi_provider import create_multi_provider_web3
from eth_defi.uniswap_v3.deployment import fetch_deployment
from eth_account import Account
from eth_account.signers.local import LocalAccount
from web3.middleware import construct_sign_and_send_raw_middleware
from eth_defi.token import fetch_erc20_details
from eth_defi.uniswap_v3.swap import swap_with_slippage_protection
from eth_defi.confirmation import wait_transactions_to_complete
import datetime
from eth_defi.revert_reason import fetch_transaction_revert_reason
from web3 import Web3

# Constants
QUOTE_TOKEN_ADDRESS = "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14"  # WETH (native)
BASE_TOKEN_ADDRESS = "0xf08A50178dfcDe18524640EA6618a1f965821715"  # USDC

# Connect to the Sepolia testnet
web3 = Web3(Web3.HTTPProvider('https://sepolia.infura.io/v3/00de74a63d9043f994b7cf455ee1e5af'))
print(f"Connected to blockchain, chain id is {web3.eth.chain_id}. The latest block is {web3.eth.block_number:,}")

# Fetch Uniswap V3 deployment details
uniswap_v3 = fetch_deployment(
    web3,
    factory_address="0x0227628f3F023bb0B980b67D528571c95c6DaC1c",
    router_address="0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E",
    position_manager_address="0x1238536071E1c677A632429e3655c799b22cDA52",
    quoter_address="0xEd1f6473345F45b75F8179591dd5bA1888cf2FB3",
)
print(f"Using Uniswap v3 compatible router at {uniswap_v3.swap_router.address}")

# Read and setup a local private key
os.environ['PRIVATE_KEY'] = '6eec9ef4c8f62e5a3432ee8dcded3da0455cdf4d2bc5b6c0d4028273b6c7d5af'
private_key = os.getenv('PRIVATE_KEY')
private_key = "0x" + private_key if not private_key.startswith("0x") else private_key
assert private_key is not None, "You must set PRIVATE_KEY environment variable"
assert private_key.startswith("0x"), "Private key must start with 0x hex prefix"
account: LocalAccount = Account.from_key(private_key)
my_address = '0x6B0b3511756f11955ca6AA69d82790E900c58D07'
assert account.address == my_address, "Account address does not match the provided MetaMask address"
print("Your MetaMask wallet address is:", my_address)

# Enable eth_sendTransaction using this private key
web3.middleware_onion.add(construct_sign_and_send_raw_middleware(account))

# Read on-chain ERC-20 token data (name, symbol, etc.)
base = fetch_erc20_details(web3, BASE_TOKEN_ADDRESS)
quote = fetch_erc20_details(web3, QUOTE_TOKEN_ADDRESS)

# Native token balance
gas_balance = web3.eth.get_balance(account.address)

print(f"Your address is {my_address}")
print(f"You have {base.fetch_balance_of(my_address)} {base.symbol}")
print(f"You have {quote.fetch_balance_of(my_address)} {quote.symbol}")
print(f"You have {gas_balance / (10 ** 18)} ETH for gas fees")

assert quote.fetch_balance_of(my_address) > 0, f"Cannot perform swap, as you have zero {quote.symbol} needed to swap"

# Ask for transfer details
decimal_amount = input(f"How many {quote.symbol} tokens you wish to swap to {base.symbol}? ")

# Input validation
try:
    decimal_amount = Decimal(decimal_amount)
except (ValueError, decimal.InvalidOperation) as e:
    raise AssertionError(f"Not a valid decimal amount: {decimal_amount}") from e

# Confirm the swap amount
print(f"Confirm swap amount {decimal_amount} {quote.symbol} to {base.symbol}")
confirm = input("Ok [y/n]?")
if not confirm.lower().startswith("y"):
    print("Aborted")
    sys.exit(1)

# Convert the amount to raw value
raw_amount = quote.convert_to_raw(decimal_amount)

# Approve the Uniswap V3 router to spend tokens
approve = quote.contract.functions.approve(uniswap_v3.swap_router.address, raw_amount)
tx_1 = approve.build_transaction({
    "gas": 850_000,
    "from": my_address,
})
print("Approve transaction:", tx_1)

# Prepare the swap transaction
bound_solidity_func = swap_with_slippage_protection(
    uniswap_v3,
    recipient_address=my_address,
    base_token=base,
    quote_token=quote,
    max_slippage=20,
    amount_in=raw_amount,
    pool_fees=[500],  # 5 BPS pool WETH-USDC
    deadline=int((datetime.datetime.now() + datetime.timedelta(minutes=10)).timestamp())
)

tx_2 = bound_solidity_func.build_transaction({
    "gas": 1_000_000,
    "from": my_address,
})

# Sign and broadcast the transactions
tx_hash_1 = web3.eth.send_transaction(tx_1)
tx_hash_2 = web3.eth.send_transaction(tx_2)

# Wait for transaction completion
tx_wait_minutes = 2.5
print(f"Broadcasted transactions {tx_hash_1.hex()}, {tx_hash_2.hex()}, now waiting {tx_wait_minutes} minutes for them to be included in a new block")
print(f"View your transactions confirming at https://sepolia.etherscan.io/address/{my_address}")

receipts = wait_transactions_to_complete(web3, [tx_hash_1, tx_hash_2], max_timeout=datetime.timedelta(minutes=tx_wait_minutes), confirmation_block_count=1)

for completed_tx_hash, receipt in receipts.items():
    if receipt["status"] == 0:
        revert_reason = fetch_transaction_revert_reason(web3, completed_tx_hash)
        raise AssertionError(f"Transaction {completed_tx_hash.hex()} failed because of: {revert_reason}")

print("All transactions successful!")
print(f"After swap, you have {base.fetch_balance_of(my_address)} {base.symbol}")
print(f"After swap, you have {quote.fetch_balance_of(my_address)} {quote.symbol}")
print(f"After swap, you have {gas_balance / (10 ** 18)} ETH left for gas fees")


Connected to blockchain, chain id is 11155111. The latest block is 6,030,730
Using Uniswap v3 compatible router at 0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E
Your MetaMask wallet address is: 0x6B0b3511756f11955ca6AA69d82790E900c58D07
Your address is 0x6B0b3511756f11955ca6AA69d82790E900c58D07
You have 0 USDC
You have 0.15 WETH
You have 0.5319647180725285 ETH for gas fees
How many WETH tokens you wish to swap to USDC? 0.01
Confirm swap amount 0.01 WETH to USDC
Ok [y/n]?y
Approve transaction: {'value': 0, 'maxFeePerGas': 27247605104, 'maxPriorityFeePerGas': 1000000000, 'chainId': 11155111, 'gas': 850000, 'from': '0x6B0b3511756f11955ca6AA69d82790E900c58D07', 'to': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14', 'data': '0x095ea7b30000000000000000000000003bfa4769fb09eefc5a80d6e87c3b9c650f7ae48e000000000000000000000000000000000000000000000000002386f26fc10000'}


ContractLogicError: execution reverted

In [27]:
import decimal
import os
import sys
from decimal import Decimal
from eth_defi.provider.multi_provider import create_multi_provider_web3
from eth_defi.uniswap_v3.deployment import fetch_deployment
from eth_account import Account
from eth_account.signers.local import LocalAccount
from web3.middleware import construct_sign_and_send_raw_middleware
from eth_defi.token import fetch_erc20_details
from eth_defi.uniswap_v3.swap import swap_with_slippage_protection
from eth_defi.confirmation import wait_transactions_to_complete
import datetime
from eth_defi.revert_reason import fetch_transaction_revert_reason
from web3 import Web3

# Constants
QUOTE_TOKEN_ADDRESS = "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14"  # WETH (native)
BASE_TOKEN_ADDRESS = "0xf08A50178dfcDe18524640EA6618a1f965821715"  # USDC

# Connect to the Sepolia testnet
web3 = Web3(Web3.HTTPProvider('https://sepolia.infura.io/v3/00de74a63d9043f994b7cf455ee1e5af'))
print(f"Connected to blockchain, chain id is {web3.eth.chain_id}. The latest block is {web3.eth.block_number:,}")

# Fetch Uniswap V3 deployment details
uniswap_v3 = fetch_deployment(
    web3,
    factory_address="0x0227628f3F023bb0B980b67D528571c95c6DaC1c",
    router_address="0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E",
    position_manager_address="0x1238536071E1c677A632429e3655c799b22cDA52",
    quoter_address="0xEd1f6473345F45b75F8179591dd5bA1888cf2FB3",
)
print(f"Using Uniswap v3 compatible router at {uniswap_v3.swap_router.address}")

# Read and setup a local private key
os.environ['PRIVATE_KEY'] = '6eec9ef4c8f62e5a3432ee8dcded3da0455cdf4d2bc5b6c0d4028273b6c7d5af'
private_key = os.getenv('PRIVATE_KEY')
private_key = "0x" + private_key if not private_key.startswith("0x") else private_key
assert private_key is not None, "You must set PRIVATE_KEY environment variable"
assert private_key.startswith("0x"), "Private key must start with 0x hex prefix"
account: LocalAccount = Account.from_key(private_key)
my_address = '0x6B0b3511756f11955ca6AA69d82790E900c58D07'
assert account.address == my_address, "Account address does not match the provided MetaMask address"
print("Your MetaMask wallet address is:", my_address)

# Enable eth_sendTransaction using this private key
web3.middleware_onion.add(construct_sign_and_send_raw_middleware(account))

# Read on-chain ERC-20 token data (name, symbol, etc.)
base = fetch_erc20_details(web3, BASE_TOKEN_ADDRESS)
quote = fetch_erc20_details(web3, QUOTE_TOKEN_ADDRESS)

# Native token balance
gas_balance = web3.eth.get_balance(account.address)

print(f"Your address is {my_address}")
print(f"You have {base.fetch_balance_of(my_address)} {base.symbol}")
print(f"You have {quote.fetch_balance_of(my_address)} {quote.symbol}")
print(f"You have {gas_balance / (10 ** 18)} ETH for gas fees")

assert quote.fetch_balance_of(my_address) > 0, f"Cannot perform swap, as you have zero {quote.symbol} needed to swap"

# Ask for transfer details
decimal_amount = input(f"How many {quote.symbol} tokens you wish to swap to {base.symbol}? ")

# Input validation
try:
    decimal_amount = Decimal(decimal_amount)
except (ValueError, decimal.InvalidOperation) as e:
    raise AssertionError(f"Not a valid decimal amount: {decimal_amount}") from e

# Confirm the swap amount
print(f"Confirm swap amount {decimal_amount} {quote.symbol} to {base.symbol}")
confirm = input("Ok [y/n]?")
if not confirm.lower().startswith("y"):
    print("Aborted")
    sys.exit(1)

# Convert the amount to raw value
raw_amount = quote.convert_to_raw(decimal_amount)

# Approve the Uniswap V3 router to spend tokens
approve = quote.contract.functions.approve(uniswap_v3.swap_router.address, raw_amount)
tx_1 = approve.build_transaction({
    "gas": 850_000,
    "from": my_address,
})
print("Approve transaction:", tx_1)

try:
    # Send the approval transaction
    tx_hash_1 = web3.eth.send_transaction(tx_1)
    print(f"Approval transaction hash: {tx_hash_1.hex()}")

    # Prepare the swap transaction
    bound_solidity_func = swap_with_slippage_protection(
        uniswap_v3,
        recipient_address=my_address,
        base_token=base,
        quote_token=quote,
        max_slippage=20,
        amount_in=raw_amount,
        pool_fees=[500],  # 5 BPS pool WETH-USDC
        deadline=int((datetime.datetime.now() + datetime.timedelta(minutes=10)).timestamp())
    )

    tx_2 = bound_solidity_func.build_transaction({
        "gas": 1_000_000,
        "from": my_address,
    })

    # Send the swap transaction
    tx_hash_2 = web3.eth.send_transaction(tx_2)
    print(f"Swap transaction hash: {tx_hash_2.hex()}")

    # Wait for transaction completion
    tx_wait_minutes = 2.5
    print(f"Broadcasted transactions {tx_hash_1.hex()}, {tx_hash_2.hex()}, now waiting {tx_wait_minutes} minutes for them to be included in a new block")
    print(f"View your transactions confirming at https://sepolia.etherscan.io/address/{my_address}")

    receipts = wait_transactions_to_complete(web3, [tx_hash_1, tx_hash_2], max_timeout=datetime.timedelta(minutes=tx_wait_minutes), confirmation_block_count=1)

    for completed_tx_hash, receipt in receipts.items():
        if receipt["status"] == 0:
            revert_reason = fetch_transaction_revert_reason(web3, completed_tx_hash)
            raise AssertionError(f"Transaction {completed_tx_hash.hex()} failed because of: {revert_reason}")

    print("All transactions successful!")
    print(f"After swap, you have {base.fetch_balance_of(my_address)} {base.symbol}")
    print(f"After swap, you have {quote.fetch_balance_of(my_address)} {quote.symbol}")
    print(f"After swap, you have {gas_balance / (10 ** 18)} ETH left for gas fees")

except Exception as e:
    print(f"An error occurred: {e}")


Connected to blockchain, chain id is 11155111. The latest block is 6,030,745
Using Uniswap v3 compatible router at 0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E
Your MetaMask wallet address is: 0x6B0b3511756f11955ca6AA69d82790E900c58D07
Your address is 0x6B0b3511756f11955ca6AA69d82790E900c58D07
You have 0 USDC
You have 0.15 WETH
You have 0.5319647180725285 ETH for gas fees
How many WETH tokens you wish to swap to USDC? 0.01
Confirm swap amount 0.01 WETH to USDC
Ok [y/n]?y
Approve transaction: {'value': 0, 'maxFeePerGas': 23943005414, 'maxPriorityFeePerGas': 1000000000, 'chainId': 11155111, 'gas': 850000, 'from': '0x6B0b3511756f11955ca6AA69d82790E900c58D07', 'to': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14', 'data': '0x095ea7b30000000000000000000000003bfa4769fb09eefc5a80d6e87c3b9c650f7ae48e000000000000000000000000000000000000000000000000002386f26fc10000'}
Approval transaction hash: 0x7f4a961941103682f4410e33d7ca97a9e66b1bbd705ee64779b81a9e2e0ba4db
An error occurred: execution reverted
